In [1]:
import csv
import json
import math
import numpy as np

from plotly import graph_objects as go

In [2]:
import os
import sys
from pathlib import Path

CWD = Path(os.getcwd())
sys.path.append(str(CWD.parent))
from model.trie import Trie

In [3]:
GUESS = 'slate'

WORD_LIST_LOC = '../data/word_list.csv'
GUESS_PATTERNS_LOC = '../data/patterns.csv'
WORDLE_TRIE_LOG = '../data/word_list_trie.json'

In [4]:
def read_csv(csv_file):
    with open(csv_file) as f:
        reader = csv.reader(f, delimiter='\n')
        return [row[0] for row in reader]

WORD_LIST = read_csv(WORD_LIST_LOC)
TOTAL_WORDS_COUNT = len(WORD_LIST)

GUESS_PATTERNS = read_csv(GUESS_PATTERNS_LOC)

In [5]:
WORD_TRIE = Trie()
with open(WORDLE_TRIE_LOG) as f:
    WORD_TRIE.root = json.load(f)

In [6]:
information = np.zeros(len(GUESS_PATTERNS))
probabilities = np.zeros(len(GUESS_PATTERNS))

for idx, guess_pattern in enumerate(GUESS_PATTERNS):
    possible_words = WORD_TRIE.find_words(GUESS, guess_pattern)
    
    probabilities[idx] = len(possible_words) / TOTAL_WORDS_COUNT
    if probabilities[idx]:
        information[idx] = -math.log(probabilities[idx], 2)
    
    WORD_TRIE.reset_pos_info()

In [7]:
information_gain_parts = probabilities * information
information_gain = sum(information_gain_parts)

In [8]:
fig = go.Figure()

x_axis = list(range(len(GUESS_PATTERNS)))
fig.add_trace(go.Bar(
    x=x_axis,
    y=sorted(information_gain_parts, reverse=True)
))

fig.update_layout(
    title_text='P(x) * I(x)',
    template='plotly_dark',
    margin=dict(l=50, r=50, t=50, b=50),
)
fig.show()